In [138]:
%load_ext autoreload
%autoreload 2
import tensorflow_quantum as tfq
from utilities.variational import VQE
from utilities.qmodels import *
from utilities.evaluator import Evaluator
from utilities.idinserter import IdInserter
from utilities.unitary_killer import UnitaryMurder
from utilities.simplifier import Simplifier
import cirq

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [143]:

vqe_handler = VQE(n_qubits=2, problem_config={"problem":"XXZ","g":1,"J":2})
iid_inserter = IdInserter(n_qubits = vqe_handler.n_qubits, selector_temperature=40, epsilon=0.1)
simplifier = Simplifier(n_qubits=vqe_handler.n_qubits)

indexed_circuit = [vqe_handler.number_of_cnots + k for k in range(vqe_handler.n_qubits, 2*vqe_handler.n_qubits )]
circuit, symbols, index_to_symbols = vqe_handler.give_circuit(indexed_circuit)
energy, resolver, history = vqe_handler.vqe(indexed_circuit)

Mindexed_circuit, Mresolver, Mindex_to_symbols = iid_inserter.place_identities(indexed_circuit,symbol_to_value=resolver,rate_iids_per_step=4)
Mcircuit = vqe_handler.give_unitary(Mindexed_circuit, Mresolver)

model = QNN(symbols=list(Mresolver.keys()), observable=vqe_handler.observable)
print(model(circuit), model(Mcircuit))

OMenergy, OMresolver, OMhistory = vqe_handler.vqe(Mindexed_circuit, symbols_to_values=Mresolver)
OMcircuit = vqe_handler.give_unitary(Mindexed_circuit, OMresolver)

#print(model(Mcircuit), model(OMcircuit))


simplifier = Simplifier(n_qubits=vqe_handler.n_qubits)
Sindexed_circuit, Sresolver, Sindex_to_symbols = simplifier.simplify_step(Mindexed_circuit, OMresolver, Mindex_to_symbols)
Scircuit = vqe_handler.give_unitary(Sindexed_circuit,Sresolver)
indexeds=[]
resolvers=[]
pps=[]
for it in range(10):
    print("****")
    e1 = model(Scircuit)
    indexeds.append(Sindexed_circuit)
    resolvers.append(Sresolver)
    pps.append(Sindex_to_symbols)
    Sindexed_circuit, Sresolver, Sindex_to_symbols = simplifier.simplify_step(Sindexed_circuit, Sresolver, Sindex_to_symbols )
    Scircuit = vqe_handler.give_unitary(Sindexed_circuit,Sresolver)
    e2 = model(Scircuit)
    if np.abs(e2-OMenergy) > 1e-5:
        print("HORROR")
        break

tf.Tensor([-0.42301735], shape=(1,), dtype=float32) tf.Tensor([-3.7217255], shape=(1,), dtype=float32)
8
3
7
8
3
8
7
7
5
5
6
7
8
5
8
7
8
****
6
4
7
HORROR


In [182]:
simplifier.testing=True
Sindexed_circuit, Sresolver, Sindex_to_symbols = simplifier.simplify_step(Mindexed_circuit, OMresolver, Mindex_to_symbols)

7
cnt_rule 7
2
cnt_rule 2
6
cnt_rule 6
7
cnt_rule 7
2
cnt_rule 2
7
cnt_rule 7
6
cnt_rule 6
6
cnt_rule 6
4
cnt_rule 4
4
cnt_rule 4
5
cnt_rule 5
6
cnt_rule 6
7
cnt_rule 7
4
cnt_rule 4
7
cnt_rule 7
6
cnt_rule 6
7
cnt_rule 7


In [183]:
model(vqe_handler.give_unitary(Sindexed_circuit, Sresolver))

<tf.Tensor: shape=(1,), dtype=float32, numpy=array([-7.5485353], dtype=float32)>

In [192]:
print(Mindexed_circuit)

[4, 1, 1, 3, 3, 5, 3, 5, 3, 4, 2, 4, 1, 3, 1, 3, 5, 0, 2, 4, 2, 5, 3, 5, 0, 3, 4, 2, 0, 2, 4, 2, 5, 3, 5, 0, 4, 1, 1, 3, 5, 3, 4, 2, 4, 1, 5, 3, 4, 2, 4, 3, 1, 0, 2, 4, 2, 5, 3, 5, 0, 3, 0, 2, 4, 2, 5, 3, 5, 0, 5, 3, 4, 2, 4, 1, 5, 3, 1, 2, 4, 2, 0, 2, 4, 2, 5, 3, 2, 4, 2, 1, 3, 5, 3, 4, 2, 4, 1, 5, 0, 5]


In [163]:
OMenergy

<tf.Tensor: shape=(), dtype=float32, numpy=-8.000006>

In [176]:
for k in range(3):
    if isinstance(k,int):
        if k==1:
            if 1+1 == 2:
                if "c" != "ac":

                    print(k)
                    break
        else:
            break

In [154]:
Scircuit = vqe_handler.give_unitary(Sindexed_circuit,Sresolver)
e2 = model(Scircuit)
e2

<tf.Tensor: shape=(1,), dtype=float32, numpy=array([-5.534333], dtype=float32)>

In [155]:
vqe_handler.give_unitary(indexeds[0], resolvers[0],)

(0, 0): ───Rx(0.951π)────Rz(-0.037π)───Rx(0.025π)─────────────────X───────────────X────────────────@───Rz(0.029π)────Rx(0.045π)───Rz(0.002π)───@───Rx(0.026π)────Rz(-0.057π)───@───Rx(0.026π)───Rz(-0.022π)─────────────────@───Rx(-0.066π)───Rz(-0.007π)───Rx(0.005π)────X───Rz(-0.02π)───Rx(0.032π)───X───@───Rz(-0.023π)───Rx(0.069π)───Rz(-0.109π)───@────────────────@───Rx(0.047π)────Rz(0.003π)──────────────────@───Rx(-0.041π)───Rz(-0.043π)───Rx(0.013π)───X─────────────────────────────X───Rz(0.083π)───Rx(0.027π)───Rz(0.109π)───@───Rx(0.097π)───Rz(0.17π)─────Rx(0.097π)───Rz(0.136π)───X───Rx(0.082π)───Rz(-0.157π)───Rx(-0.021π)───X────────────────@───
                                                                  │               │                │                                           │                               │                                            │                                             │                             │   │                                            │                │                                             │                                            │                             │                                          │                                                        │                                            │                │
(0, 1): ───Rz(-0.064π)───Rz(1.01π)─────Rx(0.05π)────Rz(-0.986π)───@───Rz(0.02π)───@───Rx(-0.03π)───X───Rz(-0.052π)───Rx(0.055π)────────────────X───Rz(-0.024π)─────────────────X───Rx(0.019π)───Rz(-0.012π)───Rx(-0.041π)───X───Rz(-0.016π)───Rx(0.016π)────Rz(-0.002π)───@───Rx(0.017π)───Rz(0.058π)───@───X───Rx(0.004π)────Rz(0.101π)───Rx(0.034π)────X───Rz(0.044π)───X───Rx(-0.023π)───Rz(-0.003π)───Rx(-0.024π)───X───Rz(-0.03π)───────────────────────────────@───Rx(0.026π)───Rz(0.074π)───@──────────────────────────────────────────X───Rx(0.156π)───Rz(-0.048π)─────────────────────────────@───Rx(0.037π)───Rz(0.071π)──────────────────@───Rx(0.007π)───X───

In [156]:
Scircuit

(0, 0): ───Rx(0.951π)───Rz(-0.037π)───Rx(0.025π)────X───X────────────────@───Rz(0.029π)────Rx(0.045π)───Rz(0.002π)───@───Rx(0.026π)────Rz(-0.057π)───@───Rx(0.026π)───Rz(-0.022π)─────────────────@───Rx(-0.066π)───Rz(-0.007π)───Rx(0.005π)────X───Rz(-0.02π)───Rx(0.032π)───X───@───Rz(-0.023π)───Rx(0.069π)───Rz(-0.109π)───@────────────────@───Rx(0.047π)────Rz(0.003π)──────────────────@───Rx(-0.041π)───Rz(-0.043π)───Rx(0.013π)───X─────────────────────────────X───Rz(0.083π)───Rx(0.027π)───Rz(0.109π)───@───Rz(0.089π)───Rx(0.188π)────Rz(0.225π)───X───Rx(0.17π)────Rz(0.097π)───Rx(0.082π)───X────────────────@───
                                                    │   │                │                                           │                               │                                            │                                             │                             │   │                                            │                │                                             │                                            │                             │                                          │                                           │                                          │                │
(0, 1): ───Rz(1.01π)────Rx(0.05π)─────Rz(-0.967π)───@───@───Rx(-0.03π)───X───Rz(-0.052π)───Rx(0.055π)────────────────X───Rz(-0.024π)─────────────────X───Rx(0.019π)───Rz(-0.012π)───Rx(-0.041π)───X───Rz(-0.016π)───Rx(0.016π)────Rz(-0.002π)───@───Rx(0.017π)───Rz(0.058π)───@───X───Rx(0.004π)────Rz(0.101π)───Rx(0.034π)────X───Rz(0.044π)───X───Rx(-0.023π)───Rz(-0.003π)───Rx(-0.024π)───X───Rz(-0.03π)───────────────────────────────@───Rx(0.026π)───Rz(0.074π)───@──────────────────────────────────────────X───Rx(0.156π)───Rz(-0.048π)────────────────@───Rx(0.037π)───Rz(0.071π)────────────────@───Rx(0.007π)───X───

In [157]:
model(Scircuit)

<tf.Tensor: shape=(1,), dtype=float32, numpy=array([-5.534333], dtype=float32)>

In [150]:
OMcircuit

(0, 0): ───Rx(0.991π)───X───X───Rx(-0.041π)───Rz(-0.037π)───Rx(0.025π)─────────────────────────────────────────X─────────────────X─────────────────────────────@───Rz(0.029π)───Rx(0.045π)────Rz(0.002π)───@───Rx(0.026π)────Rz(0.003π)───@───Rz(-0.06π)────Rx(0.026π)────Rz(-0.022π)───@───Rx(-0.033π)───X───X───Rx(-0.033π)───Rz(-0.007π)───Rx(0.034π)───X───Rx(-0.03π)───Rz(-0.02π)───Rx(0.032π)───X───@───Rz(-0.023π)───Rx(0.069π)───Rz(-0.055π)───@─────────────────@───Rz(-0.055π)───Rx(0.047π)───Rz(0.003π)───@───Rx(-0.041π)───Rz(-0.043π)───Rx(0.013π)───X──────────────────────────────X───Rz(0.083π)───Rx(0.027π)───Rz(0.023π)───@───Rz(0.086π)───Rx(0.097π)───Rz(0.053π)───Rz(0.117π)───Rx(0.097π)───Rz(0.136π)───X───Rx(0.082π)───Rz(-0.157π)───Rx(-0.021π)───X─────────────────@─────────────────
                        │   │                                                                                  │                 │                             │                                           │                              │                                             │                 │   │                                            │                                          │   │                                            │                 │                                           │                                            │                              │                                          │                                                                                 │                                            │                 │
(0, 1): ────────────────@───@───Rz(-0.032π)───Rz(-0.032π)───Rx(-0.02π)───Rz(0.02π)───Rx(-0.03π)───Rz(0.006π)───@───Rz(-0.058π)───@───Rz(0.006π)───Rx(0.028π)───X───Rx(0.028π)───Rz(-0.024π)───Rx(0.019π)───X───Rz(-0.012π)────────────────X───Rx(-0.041π)───Rz(-0.016π)───Rx(0.016π)────X─────────────────@───@───Rz(-0.002π)───Rx(0.017π)────Rz(0.058π)───@───Rx(0.004π)───Rz(0.082π)───Rz(0.019π)───@───X───Rx(0.034π)────Rz(0.044π)───Rx(-0.023π)───X───Rz(-0.003π)───X───Rx(-0.024π)───Rz(-0.03π)───Rx(0.045π)───X───Rx(-0.019π)───Rz(0.074π)─────────────────@───Rx(0.156π)───Rz(-0.048π)───@──────────────────────────────────────────X───Rx(0.037π)───Rz(0.036π)───────────────────────────────────────────────────────@───Rz(0.036π)───Rx(0.007π)────Rz(-0.023π)───@───Rx(-0.043π)───X───Rx(-0.011π)───

In [119]:
Scircuit

(0, 0): ───Rx(-1.03π)───Rz(1.25π)─────Rx(1.36π)───X────────────────X───Rz(1.1π)─────Rx(0.041π)───Rz(1.64π)────@───Rx(-0.042π)───Rz(1.73π)───@───Rx(-0.909π)───Rz(-1.25π)───Rx(-0.01π)───X───Rz(-0.905π)───Rx(-1.28π)───
                                                  │                │                                          │                             │                                           │
(0, 1): ───Rx(0.677π)───Rz(-0.455π)───────────────@───Rx(-0.58π)───@───Rz(0.428π)───Rx(-1.99π)───Rz(0.796π)───X───Rz(0.156π)────Rx(1.21π)───X───Rz(-0.45π)──────────────────────────────@───Rx(-0.583π)────────────────